# 1 - Building and saving deep learning models
In this notebook all the deep learning models will be built and persisted.

## Setup

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import pathlib

# sets the path to the home directory of this repository so other modules can be imported. 
root_path = os.path.split(os.getcwd())[0]
assert root_path.endswith("mask-detection"), "The root path does not end with mask-detection: " + root_path 
sys.path.insert(0, root_path)

# set the seed for reproducible results.
np.random.seed(0)
tf.random.set_seed(0)

# GPU settings
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.__version__)

2.3.0


In [2]:
# Facemask detection dataset
from datasets.facemask_dataset import FacemaskDataset
batch_size = 64
train_percentage = 0.6
validation_percentage = 0.2
test_percentage = 0.2
detection_ds_path = pathlib.Path(root_path + '/data/facemask-dataset')
detection_ds = FacemaskDataset(detection_ds_path, batch_size, 200, 200, True, train_percentage, validation_percentage, test_percentage)

# Facemask judge dataset
from datasets.facemask_net_dataset import FacemaskNetDataset
batch_size = 256
judge_ds_path = pathlib.Path(root_path + '/data/facemask-net-dataset')
judge_ds = FacemaskNetDataset(judge_ds_path, batch_size, train_percentage, validation_percentage, test_percentage)

train: 36 validation: 12 test: 12
train: 308 validation: 102 test: 102


## Building models

In [3]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

### Facemask detection CNN
A CNN which will be used for face mask detection

In [4]:
detection_model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.3),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.3),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')
])

detection_model.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=['acc'])

In [5]:
# Add a callback for earlystopping
callbacks = [EarlyStopping(patience=10, monitor='val_loss')]

detection_history = detection_model.fit(
    x=detection_ds.train_ds,
    validation_data=detection_ds.val_ds,
    epochs=100,
    validation_steps=detection_ds.val_size,
    steps_per_epoch=detection_ds.train_size,
    callbacks=callbacks
)

Epoch 1/100
36/36 [==============================] - 57s 2s/step - loss: 0.6928 - acc: 0.5213 - val_loss: 0.6925 - val_acc: 0.6810
Epoch 2/100
36/36 [==============================] - 2s 46ms/step - loss: 0.6901 - acc: 0.5573 - val_loss: 0.6875 - val_acc: 0.7318
Epoch 3/100
36/36 [==============================] - 2s 44ms/step - loss: 0.6807 - acc: 0.5907 - val_loss: 0.6720 - val_acc: 0.6849
Epoch 4/100
36/36 [==============================] - 2s 44ms/step - loss: 0.6583 - acc: 0.6549 - val_loss: 0.6379 - val_acc: 0.7148
Epoch 5/100
36/36 [==============================] - 2s 44ms/step - loss: 0.6204 - acc: 0.6918 - val_loss: 0.5864 - val_acc: 0.7578
Epoch 6/100
36/36 [==============================] - 2s 44ms/step - loss: 0.5688 - acc: 0.7357 - val_loss: 0.5303 - val_acc: 0.7760
Epoch 7/100
36/36 [==============================] - 2s 44ms/step - loss: 0.5163 - acc: 0.7643 - val_loss: 0.4874 - val_acc: 0.8372
Epoch 8/100
36/36 [==============================] - 2s 44ms/step - loss: 0.4

### Facemask Judge CNN
A CNN which will be used for judging if a face mask is correctly worn

In [5]:
judge_model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(236, activation='relu'),
    Dense(1, activation='sigmoid')
])

judge_model.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=['acc'])

In [7]:
judge_history = judge_model.fit(
    x=judge_ds.train_ds,
    validation_data=judge_ds.val_ds,
    epochs=50,
    validation_steps=judge_ds.val_size,
    steps_per_epoch=judge_ds.train_size
)

Epoch 1/100
1233/1233 [==============================] - 1295s 1s/step - loss: 0.2064 - acc: 0.9121 - val_loss: 0.0584 - val_acc: 0.9864
Epoch 2/100
1233/1233 [==============================] - 488s 396ms/step - loss: 0.0448 - acc: 0.9868 - val_loss: 0.0302 - val_acc: 0.9925
Epoch 3/100
  77/1233 [>.............................] - ETA: 2:29 - loss: 0.0430 - acc: 0.9868

KeyboardInterrupt: 

## Visualise training history

In [ ]:
def visualise_results(history):
    def plot_graph(history, metric, label):
        plt.plot(history.history[metric])
        plt.plot(history.history['val_'+metric], '')
        plt.xlabel("epochs", fontsize=15)
        plt.ylabel(label, fontsize=15)
        plt.legend([f'training {label}', f'validation {label}'])

    fig = plt.figure(figsize=(16,8))
    fig.suptitle("Training metrics", fontsize=20)
    plt.subplot(1,2,1)
    plot_graph(history, 'acc', 'accuracy')
    plt.ylim(None,1)
    plt.subplot(1,2,2)
    plot_graph(history, 'loss', 'loss')
    plt.ylim(0,None)

In [ ]:
visualise_results(detection_history)

In [ ]:
visualise_results(judge_history)

## Saving models

In [ ]:
detection_model.save(root_path + '/models/detection_cnn')
judge_model.save(root_path + '/models/judge_cnn')